In [48]:
!pip install -qU langchain-community pathway pyngrok sentence_transformers unstructured unstructured[pdf] pdfplumber pymupdf Pillow pandas tiktoken llm-app

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.27.0 requires sqlglot<25.2,>=23.6.3, but you have sqlglot 10.6.1 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
ibis-framework 9.2.0 requires sqlglot<25.7,>=23.4, but you have sqlglot 10.6.1 which is incompatible.


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 63.3 MB/s eta 0:00:00


In [3]:
! ngrok config add-authtoken 2oyvad5WHw2MTL0nUyx97i8M0ts_7WoRD7szU4fegHLhfEUSB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
# Copyright © 2024 Pathway

import asyncio
import functools
import inspect
import threading
from collections.abc import Callable
from typing import Any
import pathway as pw
import pathway.io as io

class _RunThread(threading.Thread):
    def __init__(self, coroutine):
        self.coroutine = coroutine
        self.result = None
        super().__init__()

    def run(self):
        self.result = asyncio.run(self.coroutine)


def _run_async(coroutine):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None
    if loop and loop.is_running():
        thread = _RunThread(coroutine)
        thread.start()
        thread.join()
        return thread.result
    else:
        return asyncio.run(coroutine)


def _coerce_sync(func: Callable) -> Callable:
    if asyncio.iscoroutinefunction(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return _run_async(func(*args, **kwargs))

        return wrapper
    else:
        return func


def _check_model_accepts_arg(model_name: str, provider: str, arg: str):
    from litellm import get_supported_openai_params

    supported_params = (
        get_supported_openai_params(model=model_name, custom_llm_provider=provider)
        or []
    )

    return arg in supported_params


def _extract_value(data: Any | pw.Json) -> Any:
    if isinstance(data, pw.Json):
        return data.value
    return data


def _unwrap_udf(func: pw.UDF | Callable) -> Callable:
    """Turn a Pathway UDF function into regular callable function."""
    if isinstance(func, pw.UDF):
        return func.func  # use settings applied to a UDF
    return func


def get_func_arg_names(func):
    sig = inspect.signature(func)
    return [param.name for param in sig.parameters.values()]

In [8]:
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pickle
import os
import pathway as pw
from pathway.xpacks.llm.vector_store import VectorStoreServer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import faiss
from pyngrok import ngrok
import time

@dataclass
class Document:
    id: str
    content: str
    metadata: Optional[Dict] = None

@dataclass
class Chunk:
    id: str
    doc_id: str
    content: str
    vector: np.ndarray = None
    metadata: Optional[Dict] = None

@dataclass
class Cluster:
    id: int
    center: np.ndarray
    chunks: List[Chunk]
    quantized_center: np.ndarray = None

In [11]:
# from pathway import Table
from pathway.internals.datasink import DataSink

In [12]:
# Implement a custom DataSink to collect data into a list
class ListDataSink(DataSink):
    def __init__(self):
        self.data = []

    def push_batch(self, batch):
        self.data.extend(batch)

    def finalize(self):
        pass  # No action needed for list collection

In [129]:
import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pickle
import os
import pathway as pw
from pathway.xpacks.llm.vector_store import VectorStoreServer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import faiss
from pyngrok import ngrok
import time

@dataclass
class Document:
    id: str
    content: str
    metadata: Optional[Dict] = None

@dataclass
class Chunk:
    id: str
    doc_id: str
    content: str
    vector: np.ndarray = None
    metadata: Optional[Dict] = None

@dataclass
class Cluster:
    id: int
    center: np.ndarray
    chunks: List[Chunk]
    quantized_center: np.ndarray = None

class EnhancedVectorStore:
    def __init__(
        self,
        embedding_model: str = "nomic-ai/nomic-embed-text-v1",
        n_clusters: int = 10,
        chunk_size: int = 500,
        host: str = "127.0.0.1",
        port: int = 8668,
        base_path: str = "vector_db",
        cache_dir: str = "./Cache",
        license_key: str = None
    ):
        self.embedding_model = embedding_model
        self.n_clusters = n_clusters
        self.chunk_size = chunk_size
        self.host = host
        self.port = port
        self.base_path = base_path
        self.cache_dir = cache_dir

        # Initialize components
        if license_key:
            pw.set_license_key(license_key)

        self.embeddings = SentenceTransformerEmbeddings(
            model_name=embedding_model,
            model_kwargs={"trust_remote_code": True}
        )
        self.text_splitter = CharacterTextSplitter(chunk_size=chunk_size)
        self.clusters = []

        # Create necessary directories
        os.makedirs(base_path, exist_ok=True)
        os.makedirs(cache_dir, exist_ok=True)

    class RetrieveQuerySchema(pw.Schema):
        query: str
        k: int

    def temp(self, files: List[Path]):
      """
      Args: Given a list of file paths, iterate through each document,
      read and create document chunks
      """

      assert len(files) > 0, "Files provided cannot be empty"



    def process_documents(self, documents: List[Document]) -> List[Chunk]:
        """Process documents into chunks with embeddings."""
        chunks = []

        for doc in documents:
            # Split text into chunks
            texts = self.text_splitter.split_text(doc.content)

            # Create chunks with embeddings
            for i, text in enumerate(texts):
                chunk = Chunk(
                    id=f"{doc.id}_chunk_{i}",
                    doc_id=doc.id,
                    content=text,
                    metadata=doc.metadata
                )
                # Get embedding vector
                chunk.vector = self.embeddings.embed_query(text)
                chunks.append(chunk)

        return chunks

    def create_clusters(self, chunks: List[Chunk]):
        """Create clusters from chunks using K-means."""
        vectors = np.array([chunk.vector for chunk in chunks])

        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        labels = kmeans.fit_predict(vectors)

        self.clusters = []
        for i in range(self.n_clusters):
            cluster_chunks = [chunk for j, chunk in enumerate(chunks) if labels[j] == i]
            cluster = Cluster(
                id=i,
                center=kmeans.cluster_centers_[i],
                chunks=cluster_chunks,
                quantized_center=self._quantize_vector(kmeans.cluster_centers_[i])
            )
            self.clusters.append(cluster)
            self._store_cluster(cluster)

    def _quantize_vector(self, vector: np.ndarray, bits: int = 8) -> np.ndarray:
        """Quantize vector to reduce memory footprint."""
        max_val = np.max(np.abs(vector))
        scale = (2 ** (bits - 1) - 1) / max_val
        quantized = np.round(vector * scale)
        return quantized / scale

    def _store_cluster(self, cluster: Cluster):
        """Store cluster data using FAISS."""
        index = faiss.IndexFlatL2(len(cluster.chunks[0].vector))
        vectors = np.array([chunk.vector for chunk in cluster.chunks])
        index.add(vectors)

        cluster_path = os.path.join(self.base_path, f"cluster_{cluster.id}")
        faiss.write_index(index, f"{cluster_path}.index")

        chunk_metadata = {chunk.id: chunk for chunk in cluster.chunks}
        with open(f"{cluster_path}_metadata.pkl", "wb") as f:
            pickle.dump(chunk_metadata, f)

    def start_server(self):

        # Set up ngrok tunnel
        public_url = ngrok.connect(self.port)
        print(f" * ngrok tunnel \"{public_url}\" -> \"http://{self.host}:{self.port}\"")

        # Run server
        self.run_server()

        return public_url

    def similarity_search(self, retrieval_queries) -> List[Chunk]:
        """Perform two-stage similarity search."""

        query = str(retrieval_queries["query"])
        n_results = 5


        query_vector = self.embeddings.embed_query(str(query))



        # Stage 1: Find closest clusters
        cluster_distances = []
        for cluster in self.clusters:
            distance = cosine(query_vector, cluster.quantized_center)
            cluster_distances.append((cluster, distance))

        top_clusters = sorted(cluster_distances, key=lambda x: x[1])[:3]

        # Stage 2: Search within top clusters
        results = []
        for cluster, _ in top_clusters:
            cluster_path = os.path.join(self.base_path, f"cluster_{cluster.id}")
            index = faiss.read_index(f"{cluster_path}.index")

            with open(f"{cluster_path}_metadata.pkl", "rb") as f:
                chunk_metadata = pickle.load(f)

            D, I = index.search(np.array([query_vector]), n_results)

            for idx in I[0]:
                chunk_id = list(chunk_metadata.keys())[idx]
                results.append(chunk_metadata[chunk_id])

        # Rerank results
        return self._rerank_results(results, query_vector)[:n_results]

    def _rerank_results(self, chunks: List[Chunk], query_vector: np.ndarray) -> List[Chunk]:
        """Rerank results based on cosine similarity."""
        chunk_scores = []
        for chunk in chunks:
            similarity = 1 - cosine(query_vector, chunk.vector)
            chunk_scores.append((chunk, similarity))

        reranked = sorted(chunk_scores, key=lambda x: x[1], reverse=True)
        return [chunk for chunk, _ in reranked]

    def run_server(
        self,
        threaded=False,
        **kwargs,
    ):
        """
        Builds the document processing pipeline and runs it without pw.run

        Args:
            threaded: if True, run in a thread. Else block computation
            kwargs: optional parameters (unused)
        """
        webserver = pw.io.http.PathwayWebserver(host=self.host, port=self.port, with_cors=True)

        # Helper function to set up routes
        def serve(route, schema, handler, documentation):
            queries, writer = pw.io.http.rest_connector(
                webserver=webserver,
                route=route,
                methods=("GET", "POST"),
                schema=schema,
                autocommit_duration_ms=50,
                delete_completed_queries=False,
                documentation=documentation,
            )
            writer(handler(queries))

        # Set up the retrieve endpoint
        serve(
            "/v1/retrieve",
            self.RetrieveQuerySchema,
            self.similarity_search,
            pw.io.http.EndpointDocumentation(
                summary="Do a similarity search for your query",
                description="Request the given number of documents from the "
                "realtime-maintained index.",
                method_types=("GET",),
            ),
        )

        # Run the server
        if threaded:
            import threading
            thread = threading.Thread(target=webserver.run)
            thread.daemon = True  # Allow the thread to be terminated when the main program exits
            thread.start()
            return thread
        else:
            webserver._run()



In [130]:
pw.set_license_key("4AB076-24E87E-EA755E-6ED4AD-6AFAAC-V3")

In [96]:
import pdfplumber
import fitz  # PyMuPDF
import io
import pandas as pd
from pathlib import Path

# Function to extract tables using pdfplumber
def extract_tables(pdf_bytes):
    tables = []
    with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(table)
    return tables


# Function to extract images using PyMuPDF and save them
def extract_images(pdf_bytes, pdf_path):
    images_info = []
    with fitz.open(stream=pdf_bytes, filetype="pdf") as doc:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            image_list = page.get_images(full=True)
            for img_index, img in enumerate(image_list):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                # Save image to filesystem
                image_filename = f"{Path(pdf_path).stem}_page{page_num + 1}_img{img_index + 1}.{image_ext}"
                image_path = os.path.join("extracted_images", image_filename)
                os.makedirs(os.path.dirname(image_path), exist_ok=True)
                with open(image_path, "wb") as img_file:
                    img_file.write(image_bytes)
                images_info.append({
                    "page": page_num + 1,
                    "image_index": img_index + 1,
                    "image_path": image_path,
                    "image_ext": image_ext
                })
    return images_info


In [171]:

# Initialize enhanced vector store
vector_store = EnhancedVectorStore(
    embedding_model="nomic-ai/nomic-embed-text-v1",
    n_clusters=5,
    chunk_size=500,
    license_key=None
)



# # Sample documents
# documents = [
#     Document(id="1", content=gandhi, metadata={"source": "file1.txt"}),
#     Document(id="2", content=luther, metadata={"source": "file2.txt"})
# ]

data = pw.io.fs.read(
    "./data",
    format="binary",
    mode="static",
    with_metadata=True,
    autocommit_duration_ms=50
)

/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/eb02ceb48c1fdcc477ff1925c9732c379f0f0d1f/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

In [172]:
# Convert the extracted_table to a Pandas DataFrame
df = pw.debug.table_to_pandas(data)
df = df.reset_index()
df = df.drop(["index"], axis=1)

In [173]:
documents = data.select(texts=extract_texts(pw.this.data))
df2 = pw.debug.table_to_pandas(documents)

In [174]:
import json
from llm_app import extract_texts
import copy
# Process each row in df
extracted_data = []
for index, row in df.iterrows():
    print(row['_metadata'])
    pdf_path = row['_metadata']['path']
    pdf_bytes = row['data']
    # try:
    text = copy.deepcopy(df2['texts'][index])
    tables = extract_tables(pdf_bytes)
    # images_info = extract_images(pdf_bytes, pdf_path)
    extracted_data.append({
        'path': pdf_path,
        'text': text,
        'tables': tables,
        # 'images': images_info
    })
    # except Exception as e:
    #     print(f"Error processing {pdf_path}: {e}")
    #     extracted_data.append({
    #         'path': pdf_path,
    #         'text': "",
    #         'tables': [],
    #         'images': []
    #     })


{"created_at": 1732995945, "modified_at": 1732995945, "owner": "root", "path": "data/IYEP-BiswadeepPurkayastha.pdf", "seen_at": 1733000400}


<ipython-input-174-33b994bc3aef>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = copy.deepcopy(df2['texts'][index])


{"created_at": 1732989590, "modified_at": 1732989590, "owner": "root", "path": "data/threads-bugs.pdf", "seen_at": 1733000400}
{"created_at": 1732996186, "modified_at": 1732996186, "owner": "root", "path": "data/Lecture 14-15 - 12 Sep 24.pdf", "seen_at": 1733000400}
{"created_at": 1733000219, "modified_at": 1733000219, "owner": "root", "path": "data/gandhi.pdf", "seen_at": 1733000400}


In [175]:
# Convert extracted_data to pandas DataFrame
extracted_df = pd.DataFrame(extracted_data)


In [176]:
# Serialize 'tables' and 'images' columns to JSON strings
extracted_df['tables'] = extracted_df['tables'].apply(lambda x: json.dumps(x))
# extracted_df['images'] = extracted_df['images'].apply(lambda x: json.dumps(x))


In [177]:
# Save extracted_df to CSV
extracted_df.to_csv('extracted_data.csv', index=False)


In [178]:
documents = []

for index, row in extracted_df.iterrows():
    text = row['text']
    paragraph = '\n\n'.join(text)
    doc = Document(id=str(index+1), content=paragraph, metadata={"source": row['path']})
    documents.append(doc)

In [179]:

# Process documents and create clusters
chunks = vector_store.process_documents(documents)
vector_store.create_clusters(chunks)

# Start server
# public_url = vector_store.start_server()
# print(f"Server running at {public_url}")

# # Example search
# results = vector_store.similarity_search("sample query", n_results=5)
# for chunk in results:
#     print(f"Document: {chunk.doc_id}, Chunk: {chunk.id}, Content: {chunk.content[:100]}...")

In [180]:
len(chunks)

129

In [181]:
# Example search
results = vector_store.similarity_search({"query": "When is Gandhi Jayanti celebrated"})
for chunk in results:
    print(f"Document: {chunk.doc_id}, Chunk: {chunk.id}, Content: {chunk.content[:100]}...")


Document: 3, Chunk: 3_chunk_4, Content: Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and wo...
Document: 3, Chunk: 3_chunk_3, Content: Gandhi's vision of an independent India based on religious pluralism was challenged in the early 194...
Document: 3, Chunk: 3_chunk_0, Content: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī;[c] 2 October 1869 – 30 January 1948)...
Document: 3, Chunk: 3_chunk_2, Content: Assuming leadership of the Indian National Congress in 1921, Gandhi led nationwide campaigns for eas...
Document: 3, Chunk: 3_chunk_1, Content: Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple ...


In [156]:
!killall ngrok

In [ ]:
# Example search
results = vector_store.similarity_search({"query": "When is Gandhi Jayanti celebrated"})
for chunk in results:
    print(f"Document: {chunk.doc_id}, Chunk: {chunk.id}, Content: {chunk.content[:100]}...")